## Finding out the top ten best-selling video games

<p>Our objective is to determine the period of time that is commonly referred to as the "Golden Age of Video Games". We will define this period as the time when the greatest number of critically acclaimed and commercially successful games were released</p>
<p>This project aims to investigate the 400 most popular video games developed from 1977 to 2020, examining a dataset that includes sales data along with reviews by critics and users. This dataset was taken from <a href="https://www.kaggle.com/holmjason2/videogamedata">Kaggle</a>. </p>
<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Gaming platform</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Game publisher</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Game developer</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Number of copies sold (millions)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Release year</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Critic score according to Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>User score according to Metacritic</td>
</tr>
</tbody>
</table>

In [ ]:
%%sql
postgresql:///games

-- Select all information and order the results from best-selling game down to tenth best-selling

SELECT *
FROM game_sales
ORDER BY games_sold DESC
LIMIT 10;


## Missing review scores
<p>The best-selling video games were released between 1985 to 2017! </p>
<p>First, it's important to explore the limitations of our database. One big shortcoming is that there is not any reviews data for some of the games on the game_sales table. </p>

In [4]:
%%sql 

SELECT COUNT(game_sales.game)
FROM game_sales
LEFT JOIN reviews
ON game_sales.game = reviews.game
WHERE critic_score IS NULL AND
user_score IS NULL;


 * postgresql:///games
1 rows affected.


count
31


## Years that video game critics loved
<p>It looks like a little less than ten percent of the games on the game_sales table don't have any reviews data. </p>
<p>There are lots of ways to measure the best years for video games! Let's start with what the critics think. </p>

In [6]:
%%sql

SELECT g.year,
ROUND(AVG(critic_score),2) AS avg_critic_score
FROM game_sales AS g
INNER JOIN reviews AS r
ON g.game = r.game
GROUP BY year
ORDER BY avg_critic_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
1998,9.32
2020,9.20
1993,9.10
1995,9.07
2004,9.03
1982,9.00
2002,8.99
1999,8.93


## Was 1982 really that great?

<p>The range of great years according to critic reviews goes from 1982 until 2020.</p>
<p>However, The value for 1982 looks especially fishy. Maybe there weren't a lot of video games in our dataset that were released in certain years. </p>
<p>Let's update our query and find out whether 1982 really was such a great year for video games.</p>

In [8]:
%%sql 

SELECT g.year,
ROUND(AVG(critic_score),2) AS avg_critic_score,
COUNT (g.game) AS num_games
FROM game_sales AS g
INNER JOIN reviews AS r
ON g.game = r.game
GROUP BY year
HAVING COUNT(g.game) > 4
ORDER BY avg_critic_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,avg_critic_score,num_games
1998,9.32,10
2004,9.03,11
2002,8.99,9
1999,8.93,11
2001,8.82,13
2011,8.76,26
2016,8.67,13
2013,8.66,18
2008,8.63,20
2012,8.62,12


## Years that dropped off the critics' favorites list
<p> The num_games column convinces us that our new list of the critics' top games reflects years that had quite a few well-reviewed games rather than just one or two hits. But which years dropped off the list due to having four or fewer reviewed games?</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>

In [10]:
%%sql 

SELECT year, avg_critic_score
FROM top_critic_years
EXCEPT
SELECT year, avg_critic_score
FROM top_critic_years_more_than_four_games
ORDER BY avg_critic_score DESC;

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


## Years video game players loved

<p>Let's analyze the years the players loved!!</p>

In [12]:
%%sql

SELECT g.year,
AVG(user_score) AS avg_user_score,
COUNT(g.game) AS num_games
FROM game_sales AS g
INNER JOIN reviews AS r
ON g.game = r.game
GROUP BY year
HAVING COUNT(year) > 4
ORDER BY avg_user_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,avg_user_score,num_games
1997,9.50,8
1998,9.40,10
2010,9.24,23
2009,9.18,20
2008,9.03,20
1996,9.00,5
2005,8.95,13
2006,8.95,16
2000,8.80,8
1999,8.80,11


## Years that both players and critics loved
<p> We've got a list of the top ten years according to both critic reviews and user reviews. Let's find out the years that both players and critics loved. </p>

<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<p>We've also saved the results of our top user years query from the previous task into a table:</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Average of all user scores for games released in that year</td>
</tr>
</tbody>
</table>

In [14]:
%%sql 

SELECT year
FROM top_critic_years_more_than_four_games
INNER JOIN top_user_years_more_than_four_games
USING (year);


 * postgresql:///games
3 rows affected.


year
1998
2008
2002


## Sales in the best video game years
<p>Looks like we've got three years that both users and critics agreed were in the top ten! We know that critics and players liked these years, but what about video game makers?</p>

In [16]:
%%sql

SELECT year,
SUM(games_sold) AS total_games_sold
FROM game_sales
WHERE year IN (SELECT year
FROM top_critic_years_more_than_four_games
INNER JOIN top_user_years_more_than_four_games
USING (year))
GROUP BY year
ORDER BY total_games_sold DESC;

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
